In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pickle
from object_detection_fastai.helper.nms_center_distance import non_max_suppression_by_distance
from tqdm import tqdm
from pathlib import Path
import numpy as np
import pandas as pd
import math

In [3]:
cat_slides = [
                "BAL_Cat_Turnbull blue_13.svs", 
                "BAL_Cat_Turnbull blue_6.svs", 
                "BAL_Cat_Turnbull blue_1.svs", 
                "BAL_Cat_Turnbull blue_12.svs", 
                "BAL_Cat_Turnbull blue_14.svs", 
                "BAL_Cat_Turnbull blue_10.svs", 
                "BAL_Cat_Turnbull blue_2.svs",]

In [4]:
fold_1 = pickle.load(open(str("inference_results_Equine-Fold-1.p"),'rb'))
fold_2 = pickle.load(open(str("inference_results_Equine-Fold-2.p"),'rb'))
fold_3 = pickle.load(open(str("inference_results_Equine-Fold-3.p"),'rb'))

In [5]:
folds = {}
nms_thresh = 40
thresh = 0.35


for file_name in tqdm(cat_slides):
    
    boxes1 = np.array(fold_1[file_name])
    boxes2 = np.array(fold_2[file_name])
    boxes3 = np.array(fold_3[file_name])
    
    boxes = np.concatenate([boxes1, boxes2, boxes3])
    init_shape = boxes.shape
    
    scales = ((boxes[:, 2] - boxes[:, 0]) / (boxes[:, 3] - boxes[:, 1]))
    boxes = boxes[(scales < 1.1) & (scales > 0.9)]
    
    annos = non_max_suppression_by_distance(boxes, boxes[:, 5], nms_thresh)
    annos = annos[annos[:, 5] > thresh]
    final_shape = annos.shape    
    
    #print("Input: {}  Output: {}".format(init_shape, final_shape))
    
    folds[file_name] = annos

100%|██████████| 7/7 [00:30<00:00,  3.30s/it]


In [6]:
pickle.dump(folds, open(str("InferenceAnnotationsCatV2.p"), "wb"))